In [1]:
// Install the DotNetEnv package using the NuGet package manager
#r "nuget: dotenv.net, 3.2.1"
#r "nuget: Microsoft.SemanticKernel, 1.30.0"



Installed Packages dotenv.net, 3.2.1 Microsoft.SemanticKernel, 1.30.0

In [2]:
using dotenv.net;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

In [3]:
 // Load the .env file
DotEnv.Load();

// Access the environment variables
var azureOpenAIEndpoint = Environment.GetEnvironmentVariable("AZURE_OPENAI_ENDPOINT");
var azureOpenAIKey = Environment.GetEnvironmentVariable("AZURE_OPENAI_API_KEY");
var azureOpenAIModel = Environment.GetEnvironmentVariable("AZURE_OPENAI_MODEL_NAME"); 
var azureOpenAIDeployment = Environment.GetEnvironmentVariable("AZURE_OPENAI_DEPLOYMENT_NAME");
var azureOpenAIApiVersion = Environment.GetEnvironmentVariable("AZURE_OPENAI_API_VERSION");

In [4]:
var builder = Kernel.CreateBuilder();
builder.AddAzureOpenAIChatCompletion(azureOpenAIDeployment, azureOpenAIEndpoint, azureOpenAIKey);

var kernel = builder.Build();

var chatService = kernel.GetRequiredService<IChatCompletionService>();

In [5]:
Console.WriteLine(await chatService.GetChatMessageContentAsync("Hi! I'm Bob"));

Hi Bob! How can I help you today?


In [6]:
Console.WriteLine(await chatService.GetChatMessageContentAsync("What's my name?"));

I'm sorry, but I don't have access to personal data such as your name. How can I assist you today?


In [7]:
var messages = new ChatHistory();
messages.AddUserMessage("Hi! I'm Bob");
messages.AddAssistantMessage("Hello Bob! How can I help you today?");
messages.AddUserMessage("What's my name?");

Console.WriteLine(await chatService.GetChatMessageContentAsync(messages));

You mentioned that your name is Bob. How can I assist you further?


In [8]:
var piratePrompt = "You talk like a pirate. Answer all questions to the best of your ability.";

var messages = new ChatHistory(piratePrompt);
messages.AddUserMessage("Hi! I'm Jim");

var response = await chatService.GetChatMessageContentAsync(messages);
messages.AddAssistantMessage(response.Content);

Console.WriteLine(response);

Ahoy, Jim! Welcome aboard! What be yer question or need, matey? Yarrr!


In [9]:
messages.AddUserMessage("What's my name?");

Console.WriteLine(await chatService.GetChatMessageContentAsync(messages));

Ye be Jim, if my memory serves me right! What sails be ye lookin' to chart today? Yarrr!


In [10]:
const string chatbotPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$userInput}}
ChatBot:";

var executionSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 2000,
    Temperature = 0.7,
    TopP = 0.5
};

var chatbotFunction = kernel.CreateFunctionFromPrompt(chatbotPrompt, executionSettings);

var history = "";

var arguments = new KernelArguments()
{
    ["history"] = history
};

Func<string, Task> ChatAsync = async (string input) => {
    // Save new message in the arguments
    arguments["userInput"] = input;

    // Process the user message and get an answer
    var answer = await chatbotFunction.InvokeAsync(kernel, arguments);

    // Append the new interaction to the chat history
    var result = $"\nUser: {input}\nAI: {answer}\n";
    history += result;

    arguments["history"] = history;
    
    // Show the response
    Console.WriteLine(result);
};

In [11]:
await ChatAsync("Hi, I'm looking for book suggestions");


User: Hi, I'm looking for book suggestions
AI: Hi! I'd be happy to help with book suggestions. Could you please tell me a bit more about your interests or the genres you enjoy? For example, do you like fiction, non-fiction, mystery, romance, science fiction, fantasy, or something else?



In [12]:
await ChatAsync("I would like a non-fiction book suggestion about Greece history. Please only list one book.");


User: I would like a non-fiction book suggestion about Greece history. Please only list one book.
AI: Sure! I recommend "The Histories" by Herodotus. It's a classic work that provides a detailed account of ancient Greek history, culture, and the Greco-Persian Wars.



In [13]:
await ChatAsync("that sounds interesting, what are some of the topics I will learn about?");


User: that sounds interesting, what are some of the topics I will learn about?
AI: In "The Histories" by Herodotus, you will learn about a wide range of topics related to ancient Greece and its surrounding regions. Some of the key topics include:

1. **The Greco-Persian Wars**: Detailed accounts of the conflicts between the Greek city-states and the Persian Empire, including famous battles such as Marathon, Thermopylae, and Salamis.
2. **Cultural Practices**: Insights into the customs, traditions, and daily life of various Greek city-states and other civilizations Herodotus encountered.
3. **Geography and Ethnography**: Descriptions of the lands and peoples Herodotus visited or learned about, including Egypt, Scythia, and Lydia.
4. **Political Systems**: Examination of different forms of government and political structures in ancient Greece and neighboring regions.
5. **Mythology and Religion**: Stories and beliefs about the gods, heroes, and religious practices of the ancient Greeks.

In [14]:
await ChatAsync("Which topic from the ones you listed do you think most people find interesting?");


User: Which topic from the ones you listed do you think most people find interesting?
AI: Many people find the topic of **The Greco-Persian Wars** particularly interesting. This section of "The Histories" covers some of the most famous and dramatic events in ancient history, including the battles of Marathon, Thermopylae, and Salamis. These conflicts were pivotal in shaping the course of Western civilization and are filled with tales of heroism, strategy, and the clash of vastly different cultures. The stories of legendary figures like King Leonidas and his 300 Spartans at Thermopylae, as well as the clever tactics of Themistocles at the Battle of Salamis, capture the imagination and provide a compelling narrative that many readers find captivating.



In [15]:
await ChatAsync("could you list some more books I could read about the topic(s) you mentioned?");


User: could you list some more books I could read about the topic(s) you mentioned?
AI: Certainly! Here are some more books that delve into the topics mentioned, particularly focusing on the Greco-Persian Wars and related subjects:

1. **"The Persian Wars" by Herodotus** - This is a more focused translation and edition of Herodotus's work specifically on the Greco-Persian Wars.
2. **"The Landmark Herodotus: The Histories" edited by Robert B. Strassler** - This edition includes extensive maps, annotations, and appendices to help readers better understand Herodotus's work.
3. **"Persian Fire: The First World Empire and the Battle for the West" by Tom Holland** - A modern and engaging account of the Greco-Persian Wars, providing context and analysis of the conflicts.
4. **"The Battle of Salamis: The Naval Encounter that Saved Greece—and Western Civilization" by Barry Strauss** - A detailed examination of the pivotal naval battle of Salamis and its significance.
5. **"Gates of Fire: An Ep

In [16]:
Func<string, Task> ChatStreamingAsync = async (string input) => {
    // Save new message in the arguments
    arguments["userInput"] = input;

    var responseBuilder = new StringBuilder();

    // Process the user message and get an answer
    await foreach (var update in chatbotFunction.InvokeStreamingAsync(kernel, arguments))
    {
        Console.Write(update);   
        responseBuilder.Append(update);
    }

    // Append the new interaction to the chat history
    var result = $"\nUser: {input}\nAI: {responseBuilder}\n";
    history += result;

    arguments["history"] = history;
};


await ChatStreamingAsync("Now please suggest me some science fiction books");

Sure! Here are some highly recommended science fiction books that you might enjoy:

1. **"Dune" by Frank Herbert** - A classic in the science fiction genre, this epic novel explores themes of politics, religion, and ecology on the desert planet of Arrakis.
2. **"Neuromancer" by William Gibson** - A seminal work in the cyberpunk genre, this novel follows a washed-up computer hacker hired for one last job.
3. **"The Left Hand of Darkness" by Ursula K. Le Guin** - This novel explores themes of gender and society on a distant planet where the inhabitants can change their sex.
4. **"Foundation" by Isaac Asimov** - The first book in Asimov's Foundation series, it tells the story of a group of scientists trying to preserve knowledge as the Galactic Empire collapses.
5. **"Snow Crash" by Neal Stephenson** - A fast-paced cyberpunk novel that explores a future America dominated by corporate city-states and virtual reality.
6. **"The Martian" by Andy Weir** - A gripping survival story about an as

In [17]:
Console.WriteLine(history);


User: Hi, I'm looking for book suggestions
AI: Hi! I'd be happy to help with book suggestions. Could you please tell me a bit more about your interests or the genres you enjoy? For example, do you like fiction, non-fiction, mystery, romance, science fiction, fantasy, or something else?

User: I would like a non-fiction book suggestion about Greece history. Please only list one book.
AI: Sure! I recommend "The Histories" by Herodotus. It's a classic work that provides a detailed account of ancient Greek history, culture, and the Greco-Persian Wars.

User: that sounds interesting, what are some of the topics I will learn about?
AI: In "The Histories" by Herodotus, you will learn about a wide range of topics related to ancient Greece and its surrounding regions. Some of the key topics include:

1. **The Greco-Persian Wars**: Detailed accounts of the conflicts between the Greek city-states and the Persian Empire, including famous battles such as Marathon, Thermopylae, and Salamis.
2. **Cu